In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


In [3]:
# load the data
data = pd.read_csv('processed_marginal_data.csv')

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch

# Load the data
data = pd.read_csv('processed_marginal_data_with_year_scaled.csv')

# Experiment with a longer sequence length (e.g., 336 hours for 2 weeks)
sequence_length = 336  # 2 weeks

# Prepare the input features (include new parameters) and the target price
X, y = [], []

for i in range(len(data) - sequence_length):
    features = data.iloc[i:i+sequence_length][[
        'Price1', 'Day_Sin', 'Day_Cos', 'Month_Sin', 'Month_Cos', 'Year_Scaled'
    ]].values.flatten()  # Flatten to make a single sequence of features
    X.append(features)
    y.append(data.iloc[i + sequence_length]['Price1'])  # Target is the price after the sequence

X = np.array(X)
y = np.array(y)

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert to torch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

print("Data preparation complete.")


Data preparation complete.


In [27]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [28]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.view(-1, sequence_length, 6)  # Reshape input for LSTM (batch_size, sequence_length, feature_dim)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Only take the output of the last time step
        return out


In [29]:
# Initialize both models
input_size = X_train.shape[1]  # Flattened input size for MLP
mlp_model = MLP(input_size)
lstm_model = LSTM(input_size=6, hidden_size=4, num_layers=2)  # LSTM input size is feature_dim (3)

# Define loss and optimizer for both models
criterion = nn.MSELoss()
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)


In [30]:
# Training function
def train_model(model, optimizer, epochs=100):
    best_val_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_output = model(X_val)
            val_loss = criterion(val_output, y_val)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    
    return model


In [31]:
print("Training MLP model...")
trained_mlp = train_model(mlp_model, mlp_optimizer)


Training MLP model...
Epoch [10/100], Loss: 8833.6016, Val Loss: 8632.0293
Epoch [20/100], Loss: 4835.5513, Val Loss: 4739.0796
Epoch [30/100], Loss: 4947.0073, Val Loss: 4942.9165
Epoch [40/100], Loss: 4443.1567, Val Loss: 4560.8403
Epoch [50/100], Loss: 4461.2026, Val Loss: 4544.2295
Epoch [60/100], Loss: 4409.4160, Val Loss: 4491.8379
Epoch [70/100], Loss: 4376.0215, Val Loss: 4463.5293
Epoch [80/100], Loss: 4359.3477, Val Loss: 4447.1230
Epoch [90/100], Loss: 4337.6328, Val Loss: 4421.7051
Epoch [100/100], Loss: 4315.2710, Val Loss: 4401.1050


In [32]:
print("Training LSTM model...")
trained_lstm = train_model(lstm_model, lstm_optimizer)


Training LSTM model...
Epoch [10/100], Loss: 11331.9121, Val Loss: 11569.7314
Epoch [20/100], Loss: 11317.9121, Val Loss: 11555.6260
Epoch [30/100], Loss: 11304.5000, Val Loss: 11542.1279
Epoch [40/100], Loss: 11291.3389, Val Loss: 11528.8096
Epoch [50/100], Loss: 11277.4023, Val Loss: 11514.5869


KeyboardInterrupt: 

In [11]:
# Evaluation function
def evaluate_model(model, X, y, model_name=""):
    model.eval()
    with torch.no_grad():
        predictions = model(X).numpy()
        true_values = y.numpy()
        test_loss = criterion(torch.tensor(predictions), torch.tensor(true_values))
        print(f"{model_name} Test Loss: {test_loss.item():.4f}")
        return predictions, true_values


In [12]:
# Evaluate both models on the test set
mlp_predictions, mlp_true_values = evaluate_model(trained_mlp, X_test, y_test, "MLP")
lstm_predictions, lstm_true_values = evaluate_model(trained_lstm, X_test, y_test, "LSTM")

# Plot predictions vs. true values for both models
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(mlp_true_values, label="True Values", color='blue')
plt.plot(mlp_predictions, label="MLP Predictions", color='red')
plt.legend()
plt.title("MLP Predictions vs. True Values")
plt.xlabel("Sample Index")
plt.ylabel("Price")

plt.subplot(1, 2, 2)
plt.plot(lstm_true_values, label="True Values", color='blue')
plt.plot(lstm_predictions, label="LSTM Predictions", color='red')
plt.legend()
plt.title("LSTM Predictions vs. True Values")
plt.xlabel("Sample Index")
plt.ylabel("Price")

plt.show()


MLP Test Loss: 1513.9897


NameError: name 'trained_lstm' is not defined

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('/home/teitur/DTU/electricproject/deeplearning/model/merged_marginal_data.csv')

# Define input and output sequence lengths
sequence_length = 336  # Use 2 weeks of history for predictions
output_sequence_length = 24  # Predict the next 24 hours

# Prepare the input features and target prices
X, y = [], []

for i in range(len(data) - sequence_length - output_sequence_length):
    # Use 'Price1' and any other relevant features you have added
    features = data[['Price1', 'Hour_Sin', 'Hour_Cos']].values
    X.append(features[i:i + sequence_length])  # input sequence
    y.append(features[i + sequence_length:i + sequence_length + output_sequence_length, 0])  # output sequence of prices

X = np.array(X)  # Shape: (samples, sequence_length, features)
y = np.array(y)  # Shape: (samples, output_sequence_length)

# Normalize data
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_shape = X.shape
X = scaler_X.fit_transform(X.reshape(-1, X.shape[2])).reshape(X_shape)

scaler_y = MinMaxScaler(feature_range=(0, 1))
y = scaler_y.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# Build the LSTM model
model = Sequential([
    LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(units=output_sequence_length)  # Output layer for sequence prediction
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
epochs = 50  # Adjust based on your needs
batch_size = 32

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluate the model
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Train Loss: {train_loss:.6f}')
print(f'Test Loss: {test_loss:.6f}')


/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-11-11 21:26:19.509313: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18966528 exceeds 10% of free system memory.
2024-11-11 21:26:19.529963: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18966528 exceeds 10% of free system memory.


Epoch 1/50


E0000 00:00:1731356782.496630 1396632 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-11-11 21:26:22.499239: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at cudnn_rnn_ops.cc:1769 : INVALID_ARGUMENT: Dnn is not supported
2024-11-11 21:26:22.499337: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Dnn is not supported
	 [[{{function_node __inference_one_step_on_data_5783}}{{node sequential_3_1/lstm_3_1/CudnnRNNV3}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_3_1/lstm_3_1/CudnnRNNV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/asyncio/base_events.py", line 639, in run_forever

  File "/home/teitur/miniconda3/envs/deeplearn/lib/python3.12/asyncio/base_events.py", line 1985, in _run_once

  File "/home/teitur/min

In [15]:
# Use the most recent data for prediction
recent_data = X[-1].reshape(1, sequence_length, X.shape[2])  # Shape it correctly for LSTM
predicted_sequence = model.predict(recent_data)

# Inverse transform the predictions to get actual prices
predicted_prices = scaler_y.inverse_transform(predicted_sequence).flatten()

# Print predicted prices for the next 24 hours
print("Predicted Prices for the Next 24 Hours:", predicted_prices)


IndexError: tuple index out of range

In [16]:
import matplotlib.pyplot as plt

# Predict on test data
y_test_pred = model.predict(X_test)
y_test_pred = scaler_y.inverse_transform(y_test_pred)
y_test_actual = scaler_y.inverse_transform(y_test)

# Plot predictions vs. actual values for one test example
plt.figure(figsize=(10, 5))
plt.plot(y_test_actual[0], label="Actual Prices", color="blue")
plt.plot(y_test_pred[0], label="Predicted Prices", color="red")
plt.title("Predicted vs. Actual Prices for Next 24 Hours")
plt.xlabel("Hour")
plt.ylabel("Price")
plt.legend()
plt.show()


NameError: name 'model' is not defined

In [23]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("Num GPUs Available:", len(gpus))
    print("Available GPU(s):", gpus)
else:
    print("No GPU detected.")


Num GPUs Available: 1
Available GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
